### **Introdução e contextualização**

A empresa Clamed Data Insights, especializada em soluções de análise de dados voltadas para o setor de saúde, recebeu uma base de informações de um conjunto de clínicas e hospitais. O objetivo é identificar padrões, anomalias e tendências que possam apoiar decisões estratégicas sobre qualidade de atendimento, custos e eficiência operacional.


### **Leitura do dataset e importação das bibliotecas que serão utilizadas**

In [2]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Leitura do dataset
df_dataset = pd.read_csv('healthcare_dataset.csv')

### **Limpeza e preparação dos dados**

In [3]:
# Leitura inicial dos dados
# - df_dataset.head()
# - df_dataset.info()
# - df_dataset.describe().round(2)
# - df_dataset.shape
# - (df_dataset.duplicated().sum())

# Problemas identificados na análise inicial:
# - Duplicatas encontradas
# - Valores ausentes em algumas colunas
# - Data de admissão como object em vez de datetime
# - Valores negativos em colunas que deveriam ser positivas

# Iniciando a limpeza dos dados:
# Identificando e removendo duplicatas
(df_dataset.duplicated(subset=['Name', 'Date of Admission', 'Test Results']).sum()) # Verificando as duplicatas pelo nome, data de admissão e resultados de testes
df_dataset = df_dataset.drop_duplicates(
    subset=['Name', 'Date of Admission', 'Test Results'],
    keep='first'
)

# Convertendo datas para o formato datetime
df_dataset['Date of Admission'] = pd.to_datetime(df_dataset['Date of Admission'], errors='coerce')
df_dataset['Discharge Date'] = pd.to_datetime(df_dataset['Discharge Date'], errors='coerce')

# Padronizando a coluna 'Name'
df_dataset['Name'] = df_dataset['Name'].str.title().str.strip()

# Identificando e removendo outliers via IQR em colunas numéricas
numeric_cols = ['Age', 'Billing Amount', 'Room Number']
for col in numeric_cols:
    Q1 = df_dataset[col].quantile(0.25)
    Q3 = df_dataset[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_dataset = df_dataset[(df_dataset[col] >= lower_bound) & (df_dataset[col] <= upper_bound)]

### **Enriquecimento e Feature Engineering**

In [7]:
# Criando uma tabela para classificação de idades (jovem, adulto, idoso)
def classify_age(age):
    if age < 18:
        return 'Young'
    elif 18 <= age < 60:
        return 'Adult'
    else:
        return 'Senior'

df_dataset['Age Group'] = df_dataset['Age'].apply(classify_age)

# Criando tabela de dias de internação
df_dataset['Days Admitted'] = (df_dataset['Discharge Date'] - df_dataset['Date of Admission']).dt.days
df_dataset['Days Admitted'] = df_dataset['Days Admitted'].fillna(0).astype(int)

# Criando coluna para filtrar pacientes ainda internados e pacientes com alta
df_dataset['Still Admitted'] = df_dataset['Discharge Date'].isna()


# Calculando o custo diário por paciente
df_dataset['Cost_Per_Day'] = df_dataset.apply(
    lambda row: row['Billing Amount'] / row['Days Admitted'] if row['Days Admitted'] > 0 else row['Billing Amount'],
    axis=1
)
df_dataset.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Age Group,Days Admitted,Still Admitted,Cost_Per_Day
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,Adult,2,False,9428.140653
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,Senior,6,False,5607.221214
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,Senior,15,False,1863.673072
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,Adult,30,False,1263.659414
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,Adult,20,False,711.915891
